In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from IPython.display import display, HTML

CALCULAR FECHAS

In [ ]:
def pedir_datos_usuario():
    # Preguntar si usará balances anuales o trimestrales
    while True:
        tipo_balance = input("¿Vas a utilizar balances anuales o trimestrales? (anual/trimestral): ").strip().lower()
        if tipo_balance in ['anual', 'trimestral']:
            break
        print("Por favor, responde 'anual' o 'trimestral'.")

    # Preguntar cuántos balances va a utilizar
    while True:
        try:
            cantidad = int(input(f"¿Cuántos balances {tipo_balance}es vas a utilizar?: "))
            if cantidad > 0:
                break
            else:
                print("La cantidad debe ser mayor a cero.")
        except ValueError:
            print("Ingresa un número válido.")
    
    # Preguntar la fecha del balance más actual
    while True:
        fecha_str = input("Ingresa la fecha del balance más actual (YYYY-MM-DD): ").strip()
        try:
            fecha_actual = datetime.strptime(fecha_str, "%Y-%m-%d")
            break
        except ValueError:
            print("Formato de fecha inválido. Usa YYYY-MM-DD.")

    return tipo_balance, cantidad, fecha_actual

def calcular_fechas_anteriores(tipo_balance, cantidad, fecha_actual):
    fechas = [fecha_actual]
    for i in range(1, cantidad):
        if tipo_balance == "anual":
            nueva_fecha = fecha_actual.replace(year=fecha_actual.year - i)
        elif tipo_balance == "trimestral":
            # Restar 3 meses (aprox 90 días) por cada balance anterior
            meses_a_restar = 3 * i
            # Calcular el nuevo año y mes
            mes = fecha_actual.month - meses_a_restar
            anio = fecha_actual.year
            while mes <= 0:
                mes += 12
                anio -= 1
            # Mantener el día, pero si el mes tiene menos días, ajustar
            dia = min(fecha_actual.day, [31,29 if anio%4==0 and (anio%100!=0 or anio%400==0) else 28,31,30,31,30,31,31,30,31,30,31][mes-1])
            nueva_fecha = fecha_actual.replace(year=anio, month=mes, day=dia)
        fechas.append(nueva_fecha)
    return sorted(fechas, reverse=True)

def main():
    tipo_balance, cantidad, fecha_actual = pedir_datos_usuario()
    fechas = calcular_fechas_anteriores(tipo_balance, cantidad, fecha_actual)
    print("\nFechas de los balances a analizar:")
    for i, fecha in enumerate(fechas, 1):
        print(f"{i}. {fecha.strftime('%Y-%m-%d')}")
        
def main():
    tipo_balance, cantidad, fecha_actual = pedir_datos_usuario()
    fechas = calcular_fechas_anteriores(tipo_balance, cantidad, fecha_actual)
    print("\nFechas de los balances a analizar:")
    for i, fecha in enumerate(fechas, 1):
        print(f"{i}. {fecha.strftime('%Y-%m-%d')}")
    return fechas

In [ ]:
fechas = main()

In [ ]:
# Pasar fechas a una fila de strings en orden (más reciente a más antigua)
fechas_str = [fecha.strftime("%Y-%m-%d") for fecha in fechas]
print(fechas_str)

CREAR TABLA VACIA

In [ ]:
columnas = ["Concepto"] + fechas_str  # Primer columna para los conceptos o cuentas

# Crear un DataFrame vacío con esos encabezados
tabla = pd.DataFrame(columns=columnas)

# Lista de conceptos
conceptos = [
    "Ventas",
    "EBITDA",
    "Intereses Pagados",
    "Deuda Financiera Total",
    "Efectivo y Equivalente de Efectivo",
    "Activo Corriente",
    "Pasivo Corriente",
    "Activo Total",
    "Pasivo Total",
    "Patrimonio Neto"
]

# Crear la tabla con la columna "Concepto" y los encabezados de fechas
tabla = pd.DataFrame(columns=["Concepto"] + fechas_str)

# Agregar los conceptos como filas, el resto queda vacío
tabla["Concepto"] = conceptos

display(tabla)

EXPORTAR TABLA A CARPETA "OUTPUT"

In [ ]:
# Ruta del archivo
nombre_archivo = "output/datos_financieros.xlsx"

# Exportar la tabla a Excel sin el índice
tabla.to_excel(nombre_archivo, index=False)
print(f"Tabla exportada correctamente a {nombre_archivo}")

DEBE COMPLETAR LA TABLA CON LOS DATOS FINANCIEROS DE LA EMPRESA QUE DESEA ANALIZAR Y, UNA VEZ QUE ESTÁ LISTA, COPIARLA EN LA CARPETA "INPUT"

In [ ]:
# Celda de validación para el usuario

respuesta = input("¿El archivo 'datos_financieros.xlsx' ya está listo en la carpeta 'input'? (si/ok para continuar): ").strip().lower()
while respuesta not in ["si", "ok"]:
    print("Por favor, asegurate de que el archivo esté listo y responde 'si' u 'ok' para continuar.")
    respuesta = input("¿El archivo 'datos_financieros.xlsx' ya está listo en la carpeta 'input'? (si/ok para continuar): ").strip().lower()

print("¡Perfecto! Continuemos...")

PASAMOS LA TABLA A DF

In [ ]:
# Ruta del archivo Excel en la carpeta input
nombre_archivo_input = "input/datos_financieros.xlsx"

# Leer el archivo Excel y convertirlo en un DataFrame
df = pd.read_excel(nombre_archivo_input)

# Mostrar las primeras filas para verificar que se leyó correctamente
df.head()

CÁLCULO DE RATIOS

In [ ]:
# Primero, asegurate de que el índice sea 'Concepto'
df = df.set_index("Concepto")

# Calculá los ratios para cada fecha (columna)
df.loc["Pasivo Corriente / Activo Corriente"] = df.loc["Pasivo Corriente"] / df.loc["Activo Corriente"]
df.loc["Pasivo / Activo"] = df.loc["Pasivo Total"] / df.loc["Activo Total"]
df.loc["Efectivo / Pasivo Corriente"] = df.loc["Efectivo y Equivalente de Efectivo"] / df.loc["Pasivo Corriente"]
df.loc["Apalancamiento"] = df.loc["Deuda Financiera Total"] / df.loc["Patrimonio Neto"]
df.loc["EBITDA / Intereses"] = df.loc["EBITDA"] / df.loc["Intereses Pagados"]
df.loc["Deuda Financiera Neta / EBITDA"] = (df.loc["Deuda Financiera Total"] - df.loc["Efectivo y Equivalente de Efectivo"]) / df.loc["EBITDA"]
df.loc["Deuda Financiera Neta / Patrimonio Neto"] = (df.loc["Deuda Financiera Total"] - df.loc["Efectivo y Equivalente de Efectivo"]) / df.loc["Patrimonio Neto"]

CALCULO PROMEDIO PONDERADO

In [ ]:
# Detecta todas las columnas de fechas dinámicamente (exceptuando columnas extra si hay)
fechas = df.columns.tolist()
if "Promedio Ponderado" in fechas:
    fechas.remove("Promedio Ponderado")

# Los pesos: mayor peso a la izquierda (más antigua), menor a la derecha
n = len(fechas)
pesos = np.linspace(n, 1, n)  # Ejemplo: [3,2,1] para 3 fechas
pesos = pesos / pesos.sum()   # Normaliza para que sumen 1

# Calcula el promedio ponderado de los ratios
ratios = [
    "Pasivo Corriente / Activo Corriente",
    "Pasivo / Activo",
    "Efectivo / Pasivo Corriente",
    "Apalancamiento",
    "EBITDA / Intereses",
    "Deuda Financiera Neta / EBITDA",
    "Deuda Financiera Neta / Patrimonio Neto"
]

ponderados = {}
for ratio in ratios:
    if ratio in df.index:
        valores = df.loc[ratio, fechas].astype(float)
        ponderado = np.dot(valores, pesos)
        ponderados[ratio] = ponderado

# Agrega los promedios ponderados como columna
for ratio, valor in ponderados.items():
    df.loc[ratio, "Promedio Ponderado"] = valor

# Muestra la tabla final
df

DEFINICIÓN DE NOTAS

In [ ]:
def nota_pasivo_corriente_activo_corriente(x):
    if x <= 0.4: return 10
    elif x <= 0.5: return 9
    elif x <= 0.6: return 8
    elif x <= 0.6999: return 7
    elif x <= 0.8: return 6
    elif x <= 0.85: return 5
    elif x <= 0.99: return 4
    elif x <= 1.1: return 3
    elif x <= 1.5: return 2
    else: return 1

def nota_pasivo_activo(x):
    if x <= 0.4: return 10
    elif x <= 0.5: return 9
    elif x <= 0.6: return 8
    elif x <= 0.6999: return 7
    elif x <= 0.8: return 6
    elif x <= 0.85: return 5
    elif x <= 0.99: return 4
    elif x <= 1.1: return 3
    elif x <= 1.5: return 2
    else: return 1

def nota_efectivo_pasivo_corriente(x):
    if x >= 1.5: return 10
    elif x >= 1.3: return 9
    elif x >= 1.1: return 8
    elif x >= 1: return 7
    elif x >= 0.9: return 6
    elif x >= 0.85: return 5
    elif x >= 0.8: return 4
    elif x >= 0.7: return 3
    elif x >= 0.6: return 2
    else: return 1

def nota_apalancamiento(x):
    if x <= 1: return 10
    elif x <= 1.3: return 9
    elif x <= 1.5: return 8
    elif x <= 1.99: return 7
    elif x <= 2.05: return 6
    elif x <= 2.1: return 5
    elif x <= 2.2: return 4
    elif x <= 2.3: return 3
    elif x <= 2.5: return 2
    else: return 1

def nota_ebitda_intereses(x):
    if x >= 6: return 10
    elif x >= 5: return 9
    elif x >= 4: return 8
    elif x >= 3: return 7
    elif x >= 2.5: return 6
    elif x >= 1.5: return 5
    elif x >= 1: return 4
    elif x >= 0.9: return 3
    elif x >= 0.5: return 2
    else: return 1

def nota_deuda_fin_neta_ebitda(x):
    if x <= 1: return 10
    elif x <= 2: return 9
    elif x <= 2.5: return 8
    elif x <= 2.9999: return 7
    elif x <= 3.1: return 6
    elif x <= 3.5: return 5
    elif x <= 3.9999: return 4
    elif x <= 4.1: return 3
    elif x <= 4.5: return 2
    else: return 1

def nota_deuda_fin_neta_patrimonio(x):
    if x <= 0.2: return 10
    elif x <= 0.5: return 9
    elif x <= 0.8: return 8
    elif x <= 0.9999: return 7
    elif x <= 1.1: return 6
    elif x <= 1.3: return 5
    elif x <= 1.4999: return 4
    elif x <= 1.6: return 3
    elif x <= 1.8: return 2
    else: return 1

In [ ]:
df.loc["Pasivo Corriente / Activo Corriente", "Nota"] = nota_pasivo_corriente_activo_corriente(df.loc["Pasivo Corriente / Activo Corriente", "Promedio Ponderado"])
df.loc["Pasivo / Activo", "Nota"] = nota_pasivo_activo(df.loc["Pasivo / Activo", "Promedio Ponderado"])
df.loc["Efectivo / Pasivo Corriente", "Nota"] = nota_efectivo_pasivo_corriente(df.loc["Efectivo / Pasivo Corriente", "Promedio Ponderado"])
df.loc["Apalancamiento", "Nota"] = nota_apalancamiento(df.loc["Apalancamiento", "Promedio Ponderado"])
df.loc["EBITDA / Intereses", "Nota"] = nota_ebitda_intereses(df.loc["EBITDA / Intereses", "Promedio Ponderado"])
df.loc["Deuda Financiera Neta / EBITDA", "Nota"] = nota_deuda_fin_neta_ebitda(df.loc["Deuda Financiera Neta / EBITDA", "Promedio Ponderado"])
df.loc["Deuda Financiera Neta / Patrimonio Neto", "Nota"] = nota_deuda_fin_neta_patrimonio(df.loc["Deuda Financiera Neta / Patrimonio Neto", "Promedio Ponderado"])

NOTA FINAL

In [ ]:
nota_final = (
    df.loc["Apalancamiento", "Nota"] * 0.25
    + df.loc["Deuda Financiera Neta / Patrimonio Neto", "Nota"] * 0.20
    + df.loc["Deuda Financiera Neta / EBITDA", "Nota"] * 0.15
    + df.loc["EBITDA / Intereses", "Nota"] * 0.15
    + df.loc["Efectivo / Pasivo Corriente", "Nota"] * 0.10
    + df.loc["Pasivo Corriente / Activo Corriente", "Nota"] * 0.10
    + df.loc["Pasivo / Activo", "Nota"] * 0.05
)

In [ ]:
# Definí los nombres de ratios para mostrar
ratios_nombres = [
    ("Pasivo Corriente / Activo Corriente", "Pasivo Corriente / Activo Corriente"),
    ("Pasivo / Activo", "Pasivo / Activo"),
    ("Efectivo / Pasivo Corriente", "Efectivo / Pasivo Corriente"),
    ("Apalancamiento", "Apalancamiento"),
    ("EBITDA / Intereses", "EBITDA / Intereses"),
    ("Deuda Financiera Neta / EBITDA", "Deuda Financiera Neta / EBITDA"),
    ("Deuda Financiera Neta / Patrimonio Neto", "Deuda Financiera Neta / Patrimonio Neto")
]

In [ ]:
def color_nota(nota):
    # Nota mínima = 1 (rojo), máxima = 10 (verde)
    # Interpolación entre rojo (255,0,0), amarillo (255,255,0), verde (0,128,0)
    if nota <= 3:
        r, g, b = 229, 57, 53
    elif nota <= 7:
        # Amarillo: (251, 192, 45)
        r, g, b = 251, 192, 45
    else:
        # Verde: (67, 160, 71)
        r, g, b = 67, 160, 71
    return f'rgb({r},{g},{b})'

def show_notas_color(ratios_nombres, df, nota_final):
    html = "<b>Notas de cada ratio:</b><br><br>"
    for key, nombre in ratios_nombres:
        nota = df.loc[key, "Nota"]
        color = color_nota(nota)
        html += (
            f'<div style="margin-bottom:16px;">'
            f'<span style="background-color:{color}; color:white; padding:10px 24px; border-radius:25px; font-size:16px; min-width:180px; display:inline-block;">'
            f'{nombre}: {nota}'
            f'</span>'
            f'</div>'
        )
    html += (
        f'<br><b>Nota final:</b><br>'
        f'<span style="background-color:{color_nota(nota_final)}; color:white; padding:12px 32px; border-radius:30px; font-size:20px; min-width:200px; display:inline-block;">'
        f'{nota_final:.2f}'
        f'</span>'
    )
    display(HTML(html))

show_notas_color(ratios_nombres, df, nota_final)